In [ ]:
import os
import shutil
from moviepy.editor import VideoFileClip

#비디오를 길이가 짧은 순서대로 정렬한 후 '1.mp4', '2.mp4' ... 이름으로 다시 저장하고 각 비디오의 오디오 트랙을 1.wav, 2.wav...로 추출하여 저장하는 함수
def sort_videos_by_length_and_extract_audio(input_folder, output_audio_folder, output_video_folder):
    # input_folder에서 비디오 파일 리스트 가져오기
    video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))]
    video_paths = [os.path.join(input_folder, f) for f in video_files]
    
    # 비디오 파일마다 길이 저장
    video_lengths = []
    for video_path in video_paths:
        with VideoFileClip(video_path) as video:
            video_length = video.duration
        video_lengths.append((video_path, video_length))
    
    # 비디오 길이에 따라 정렬
    sorted_videos = sorted(video_lengths, key=lambda x: x[1])
    
    # 정렬된 순서대로 파일 복사 및 이름 변경, 오디오 추출
    for i, (video_path, _) in enumerate(sorted_videos, start=1):
        # 새로운 파일 이름 설정
        new_video_name = f"{i}{os.path.splitext(video_path)[1]}"
        
        # 원본 비디오 파일을 새 위치로 복사
        new_video_path = os.path.join(output_video_folder, new_video_name)
        shutil.copy(video_path, new_video_path)
        
        # 오디오 추출
        with VideoFileClip(new_video_path) as video:
            audio_path = os.path.join(output_audio_folder, f"{i}.wav")
            video.audio.write_audiofile(audio_path)

In [ ]:
import numpy as np
from scipy.signal import correlate
from scipy.io import wavfile
import os

def cal_adjustment_time(audio_path):
    # audio_path 내의 모든 wav 파일 리스트 가져오기
    audio_files = [f for f in os.listdir(audio_path) if f.endswith('.wav')]
    audio_paths = [os.path.join(audio_path, f) for f in audio_files]

    # 기준 영상 대비 비교 영상의 앞부분이 얼마만큼 밀려있는지를 저장하는 리스트
    front_adjustment_lst = []
    back_adjustment_lst = []

    # 기준 영상의 길이 계산
    _, base_audio = wavfile.read(audio_paths[0])
    base_duration = len(base_audio) / 44100  # 기준 오디오 파일의 길이(초)
    min_base_length = base_duration
    
    # audio_path 내의 두번째 파일부터 adjusted_front_duration 계산
    for audio_path in audio_paths[1:]:
        rate, target_audio = wavfile.read(audio_path)
        target_duration = len(target_audio) / rate  # target 오디오 파일의 길이(초)

        base_waveform = base_audio  # 기준 무대 영상의 waveform
        target_waveform = target_audio  # 비교할 무대 영상의 waveform

        # waveform 정규화
        base_waveform = base_waveform / np.max(np.abs(base_waveform))
        target_waveform = target_waveform / np.max(np.abs(target_waveform))

        # 상호 상관 함수 계산
        correlation = correlate(target_waveform, base_waveform, mode='same')

        # 상호 상관 함수에서 피크값 찾기
        peak_index = np.argmax(correlation)

        
        # base_audio와 target_audio간의 adjusted_front_duration 계산
        adjusted_front_duration = ((peak_index - len(base_waveform)) // 2) / rate
        front_adjustment_lst.append(adjusted_front_duration)

        # 영상의 앞 부분을 adjusted_front_duration만큼 잘랐을 때 base_duration보다 짧아질 경우 min_base_length 갱신
        if target_duration - adjusted_front_duration < min_base_length:
            min_base_length = target_duration - adjusted_front_duration
            
    front_adjustment_lst = [0] + front_adjustment_lst #각 비디오의 앞부분 조정 시간
    
    # audio_path 내의 모든 audio 파일에 대해 adjusted_back_duration 계산
    for i, audio_path in enumerate(audio_paths):
        rate, audio_file = wavfile.read(audio_path)
        audio_duration = len(audio_file) / rate  #오디오 파일의 길이(초)
        if audio_duration - front_adjustment_lst[i] > min_base_length:
            adjusted_back_duration = audio_duration - front_adjustment_lst[i] - min_base_length
        else:
            adjusted_back_duration = 0
        back_adjustment_lst.append(adjusted_back_duration)
                     
    return front_adjustment_lst, back_adjustment_lst

In [ ]:
def trim_audio_files(output_audio_folder, front_adjustment_lst, back_adjustment_lst):
    audio_files = [f for f in os.listdir(output_audio_folder) if f.endswith('.wav')]
    audio_paths = [os.path.join(output_audio_folder, f) for f in audio_files]
    
    for i, audio_path in enumerate(audio_paths):
        rate, audio_file = wavfile.read(audio_path)

        # 앞부분 자르기
        front_trim_samples = int(front_adjustment_lst[i] * rate)
        if front_trim_samples > 0:
            audio_file = audio_file[front_trim_samples:]
        elif front_trim_samples < 0:
            padding = np.zeros(abs(front_trim_samples), dtype=audio_file.dtype)
            audio_file = np.concatenate((padding, audio_file))

        # 뒷부분 자르기
        back_trim_samples = int(back_adjustment_lst[i] * rate)
        if back_trim_samples > 0:
            audio_file = audio_file[:-back_trim_samples]

        # 결과 저장
        output_path = os.path.join(output_audio_folder, f"{os.path.basename(audio_path)}")
        wavfile.write(output_path, rate, audio_file)

### CPU 병렬처리

In [ ]:
from trim_vids_and_extract_frames2 import trim_and_extract_frames_parallel

# 폴더 경로 설정
input_folder = 'input_videos'
output_audios_folder = 'output_audios' #길이순으로 정렬된 영상에서 추출한 오디오 트랙들이 저장되는 폴더
output_videos_folder = 'output_videos' #길이순으로 정렬된 영상이 새로운 이름으로 저장되는 폴더
output_frames_folder = 'output_frames' # 추출한 프레임들이 저장될 폴더

# 프레임 추출 특성 설정
fps = 29.97 # 추출할 초당 프레임수 지정
resize_size = (1920, 1080)  # 이미지 리사이즈 크기

# 함수 실행
sort_videos_by_length_and_extract_audio(input_folder, output_audios_folder, output_videos_folder) #input_folder에 있는 영상들을 길이순으로 정렬하고 오디오 트랙 추출 및 이름 변경하여 새롭게 저장
front_adjustment_lst, back_adjustment_lst = cal_adjustment_time(output_audios_folder) #길이가 가장 짧은 무대 영상을 기준으로 했을 때 비교할 영상이 앞뒤로 얼마만큼 조정해야지를 계산
trim_audio_files(output_audios_folder, front_adjustment_lst, back_adjustment_lst) # 추출한 오디오 길이 조정
trim_and_extract_frames_parallel(output_videos_folder, output_frames_folder, front_adjustment_lst, back_adjustment_lst, resize_size, fps) #비교 영상이 기준 영상과 길이가 같고 오디오 싱크가 맞도록 조정한 후 프레임을 추출하여 저장